<a href="https://colab.research.google.com/github/awagler2/NODE/blob/main/Typology_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1

We merge institutional data from multiple CSV files into a unified dataset suitable for typology analysis. Each file contains different features about institutions, such as financial data, enrollment numbers, or instructional staff details. By combining these files, we create a comprehensive view of each institution's characteristics over multiple years.

So we start with reading all relevant CSV files and a mapping file that specifies which columns to extract from each source. For columns not explicitly mapped but important for the analysis, a list of additional features ensures their inclusion. The code handles data inconsistencies, such as duplicate columns and missing values, by cleaning and standardizing the data. Missing-like values, such as empty strings or "Missing/not reported," are replaced with "N/A" to maintain uniformity.

To enhance the dataset, metadata like the institution's name, longitude, and latitude is added from a separate directory file. After merging all the data, the columns are rearranged for clarity, ensuring key identifiers like `unitid` and `year` are upfront, followed by location information and features. Finally, the cleaned, deduplicated dataset is saved as a CSV file, ready for analysis. This approach ensures a scalable, consistent, and comprehensive integration of data from diverse sources.


In [ ]:
import pandas as pd

csv_files = [
    '/content/enrollment.csv',
    '/content/finance.csv',
    '/content/directory.csv',
    '/content/program-year-tuition-cip.csv',
    '/content/salaries-instructional-staff.csv',
    '/content/salaries-noninstructional-staff.csv',
    '/content/sfa-all-undergraduates.csv',
    '/content/sfa-by-living-arrangement.csv',
    '/content/sfa-by-tuition-type.csv',
    '/content/student-faculty-ratio.csv',
    '/content/completers.csv',
    '/content/fall-enrollment-res.csv',
    '/content/gradrates.csv',
    '/content/institutional-characteristics.csv',
    '/content/libraries.csv',
    '/content/outcome-measures.csv',
    '/content/completers.csv',
    '/content/sfa-ftft.csv',
    '/content/gradrates200.csv',
    '/content/sfa-grants-and-net-price.csv',
    '/content/requirements.csv',
    '/content/gradratespell.csv',
]

additional_features = [
    'acad_year_length_hours', 'acad_year_length_weeks', 'academic_rank', 'assets', 'assets_net',
    'athletic_expense_treatment', 'average_grant', 'average_salary', 'avg_wgtd_mon_salary',
    'award_assoc_4yr', 'award_bach_4yr', 'award_cert_4yr', 'books_supplies', 'buildings',
    'capital_assets_other', 'completers', 'completers_100pct', 'completers_150pct', 'completers_4yr',
    'completion_rate_150pct', 'completion_rate_4yr', 'conf_number_baseball', 'conf_number_basketball',
    'conf_number_football', 'conf_number_track', 'def_inflows_pension', 'def_inflows_resources',
    'def_outflows_pension', 'def_outflows_resources', 'dist_courses_offered', 'dist_ed_not_offered',
    'dist_grad_not_offered', 'dist_grad_progs_offered', 'dist_progs_offered', 'dist_ug_not_offered',
    'dist_ug_offered', 'dist_ug_progs_offered', 'dormitory_capacity', 'electronic_books',
    'electronic_databases', 'electronic_media', 'electronic_serials', 'endowment_beg', 'endowment_end',
    'enrollment', 'exp_acad_inst_student_salaries', 'exp_acad_inst_student_total', 'exp_acad_supp_salaries',
    'exp_acad_supp_total', 'exp_aux_ent_salaries', 'exp_aux_ent_total', 'exp_fringe_benefits',
    'exp_hospital_salaries', 'exp_hospital_total', 'exp_ind_op_salaries', 'exp_ind_op_total',
    'exp_inst_supp_salaries', 'exp_inst_supp_total', 'exp_instruc_salaries', 'exp_instruc_total',
    'exp_net_grant_aid_salaries', 'exp_net_grant_aid_total', 'exp_research_salaries',
    'exp_research_total', 'exp_sal_wages_from_lib_budget', 'exp_student_serv_salaries',
    'exp_student_serv_total', 'income_level', 'income_net', 'income_tax_fed', 'income_tax_state',
    'infrastructure', 'instruc_staff_count', 'intangible_assets_net', 'interlibrary_loan_services',
    'interlibrary_loans_provided', 'interlibrary_loans_received', 'invest_capital_assets',
    'land_improvements', 'liabilities', 'living_arrangement', 'longterm_debt', 'longterm_investments',
    'meals_per_week', 'military_training_credit', 'net_equity_beg_adjust', 'net_pension_liability',
    'net_position_adjustments', 'net_position_beginning', 'net_position_change', 'net_position_end',
    'number_admitted', 'number_applied', 'number_enrolled_ft', 'number_enrolled_pt',
    'number_enrolled_total', 'number_of_students', 'number_receiving_grants', 'parent_child_allocation',
    'parent_child_flag', 'parent_child_system_flag', 'parent_unitid', 'pell_grant_treatment',
    'pension_expense', 'percent_of_cohort', 'percent_of_students', 'percent_of_undergrads',
    'program_length_hours', 'program_length_weeks', 'program_size_rank', 'race', 'rev_additions',
    'rev_affiliated_entities', 'rev_appropriations_fed', 'rev_appropriations_local',
    'rev_appropriations_state', 'rev_auxiliary_enterprises_gross', 'rev_auxiliary_enterprises_net',
    'rev_capital_approps', 'rev_capital_grants_gifts', 'rev_edu_services_sales', 'rev_endowment_additions',
    'rev_endowment_income', 'rev_fed_approps_grants', 'rev_gifts_grants_contracts',
    'rev_grants_contracts_federal', 'rev_grants_contracts_local', 'rev_grants_contracts_state',
    'rev_hosp_ind_op_other', 'rev_hospital', 'rev_independent_operations', 'rev_investment_return',
    'rev_nonoperating', 'rev_operating', 'rev_other', 'rev_other_additions', 'rev_other_nonoperating',
    'rev_other_operating', 'rev_state_local_approps_grants', 'rev_total_current',
    'rev_tuition_fees_gross', 'rev_tuition_fees_net', 'room_board_charge', 'rotc_navy_marinecorps',
    'sch_allowances_aux_enterp', 'sch_allowances_total', 'sch_allowances_tuition_fees',
    'sch_exp_net_fellowships', 'sch_grants_institutional', 'sch_grants_local', 'sch_grants_private',
    'sch_grants_state', 'sch_grants_state_local', 'sch_other_federal_grants', 'sch_pell_grant',
    'sch_restricted_inst_grants', 'sch_total_student_aid', 'sch_unrestricted_inst_grants',
    'undergrad_research'
]

mappings_file = '/content/IPEDS_CSV_Variable_Mappings.csv'
mappings_df = pd.read_csv(mappings_file)

merged_data = pd.DataFrame()

def deduplicate_and_align(data, on_columns):
    return data.drop_duplicates(subset=on_columns).reset_index(drop=True)

def clean_tuition_data(file_path):
    tuition_data = pd.read_csv(file_path, low_memory=False)
    tuition_data = tuition_data.drop(columns=['Unnamed: 0'])
    tuition_data = tuition_data[tuition_data['year'] >= 2017]

    tuition_data.replace({'Missing/not reported': 'N/A', None: 'N/A'}, inplace=True)
    tuition_data.fillna({'tuition_fees': 0, 'books_supplies': 0}, inplace=True)

    aggregated_data = tuition_data.groupby(['unitid', 'year']).agg({
        'tuition_fees': 'mean',
        'books_supplies': 'mean',
        'program_length_hours': 'median',
        'contact_or_credit_hours': 'first'
    }).reset_index()

    return aggregated_data

tuition_data_cleaned = clean_tuition_data('/content/program-year-tuition-cip.csv')

processed_features = set()

for csv_file in csv_files:
    if 'program-year-tuition-cip' in csv_file:
        continue

    file_name = csv_file.split('/')[-1].replace('.csv', '')
    mapped_vars = mappings_df[mappings_df['CSV_File_Source'] == file_name]

    df = pd.read_csv(csv_file, low_memory=False)

    if df.columns.duplicated().any():
        print(f"Duplicate columns detected in {csv_file}: {df.columns[df.columns.duplicated()].tolist()}")
        df = df.loc[:, ~df.columns.duplicated()]

    df.replace({"Missing/not reported": "N/A", "": "N/A"}, inplace=True)
    df.fillna("N/A", inplace=True)

    relevant_columns = ['unitid', 'year'] + mapped_vars['Matched_CSV_Variable'].tolist()
    for feature in additional_features:
        if feature in df.columns and feature not in processed_features:
            relevant_columns.append(feature)
            processed_features.add(feature)

    relevant_columns = [col for col in relevant_columns if col in df.columns]
    if relevant_columns:
        df = df[relevant_columns]

        if 'year' in df.columns:
            df = df[df['year'] >= 2017]

        df = deduplicate_and_align(df, ['unitid', 'year'])

        if merged_data.empty:
            merged_data = df
        else:
            merged_data = pd.merge(
                merged_data,
                df,
                on=['unitid', 'year'],
                how='outer',
                suffixes=('', f'_{file_name}')
            )

merged_data = pd.merge(
    merged_data,
    tuition_data_cleaned,
    on=['unitid', 'year'],
    how='outer'
)

directory_file = '/content/directory.csv'
directory_df = pd.read_csv(directory_file, low_memory=False)

directory_df.rename(columns={
    'inst_name': 'institution_name',
    'long': 'longitude',
    'lat': 'latitude'
}, inplace=True)

required_columns = ['unitid', 'institution_name', 'longitude', 'latitude']
if all(col in directory_df.columns for col in required_columns):
    merged_data = pd.merge(
        merged_data,
        directory_df[required_columns],
        on='unitid',
        how='left'
    )
else:
    print("Directory file is missing one or more required columns.")

merged_data.replace({"": "N/A", "Missing/not reported": "N/A", "Not applicable": "N/A"}, inplace=True)
merged_data.fillna("N/A", inplace=True)

column_order = ['unitid', 'year', 'longitude', 'latitude', 'institution_name'] + \
               [col for col in merged_data.columns if col not in ['unitid', 'year', 'longitude', 'latitude', 'institution_name']]
merged_data = merged_data[column_order]

merged_data = deduplicate_and_align(merged_data, ['unitid', 'year'])

output_file = '/content/Merged_Indicators_By_Year.csv'
merged_data.to_csv(output_file, index=False)

from google.colab import files
files.download(output_file)

print(f"Merged data saved to {output_file}.")

<ipython-input-1-d60432a643ab>:129: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'N/A' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("N/A", inplace=True)
<ipython-input-1-d60432a643ab>:129: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'N/A' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("N/A", inplace=True)
<ipython-input-1-d60432a643ab>:129: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'N/A' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("N/A", inplace=True)
<ipython-input-1-d60432a643ab>:129: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an err

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Merged data saved to /content/Merged_Indicators_By_Year.csv.


# Step 2

In [ ]:
!pip install missingno

In [ ]:
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

merged_data = pd.read_csv('/content/Merged_Indicators_By_Year.csv')

def split_dataset(data, group_size=15):
    column_groups = [data.columns[i:i+group_size] for i in range(0, len(data.columns), group_size)]
    return [data[columns] for columns in column_groups]

output_pdf = PdfPages('Missing_Data_Analysis.pdf')

print("Big Matrix and Heatmap Overview for Entire Dataset")
plt.figure(figsize=(18, 8))
msno.matrix(merged_data, sparkline=False)
plt.title('Missing Data - Entire Dataset')
plt.savefig(output_pdf, format='pdf')
plt.close()

plt.figure(figsize=(18, 8))
msno.heatmap(merged_data)
plt.title('Missing Data Correlation - Entire Dataset')
plt.savefig(output_pdf, format='pdf')
plt.close()

feature_groups = split_dataset(merged_data, group_size=15)

for idx, group_data in enumerate(feature_groups):
    group_name = f"Feature Group {idx + 1}"
    print(f"\nAnalyzing missing data for {group_name}...\n")

    plt.figure(figsize=(12, 6))
    msno.matrix(group_data, sparkline=False)
    plt.title(f'Missing Data - {group_name}')
    plt.savefig(output_pdf, format='pdf')
    plt.close()

    plt.figure(figsize=(10, 6))
    msno.heatmap(group_data)
    plt.title(f'Missing Data Correlation - {group_name}')
    plt.savefig(output_pdf, format='pdf')
    plt.close()

    missing_counts = group_data.isnull().sum()
    missing_percentage = (missing_counts / len(group_data)) * 100
    summary_df = pd.DataFrame({
        'Variable': group_data.columns,
        'Missing Values': missing_counts,
        'Percentage Missing': missing_percentage
    }).sort_values(by='Missing Values', ascending=False)

    print(f"Summary of missing data for {group_name}:")
    print(summary_df)

    fig, ax = plt.subplots(figsize=(12, len(summary_df) * 0.3))
    ax.axis('tight')
    ax.axis('off')
    table = ax.table(
        cellText=summary_df.values,
        colLabels=summary_df.columns,
        loc='center',
        cellLoc='center'
    )
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.auto_set_column_width(col=list(range(len(summary_df.columns))))
    plt.title(f'Summary of Missing Data - {group_name}', fontsize=14)
    plt.savefig(output_pdf, format='pdf')
    plt.close()

output_pdf.close()

from google.colab import files
files.download('Missing_Data_Analysis.pdf')

print("Missing Data Analysis saved to Missing_Data_Analysis.pdf and downloaded.")


<ipython-input-11-de87b5b52827>:7: DtypeWarning: Columns (12,126,140,141,142,143,144) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_data = pd.read_csv('/content/Merged_Indicators_By_Year.csv')


Big Matrix and Heatmap Overview for Entire Dataset

Analyzing missing data for Feature Group 1...

Summary of missing data for Feature Group 1:
                                              Variable  Missing Values  \
capital_assets_other              capital_assets_other           39779   
assets_net                                  assets_net           38302   
buildings                                    buildings           36608   
athletic_expense_treatment  athletic_expense_treatment           36572   
assets                                          assets           36216   
number_enrolled_pt                  number_enrolled_pt           31235   
number_enrolled_ft                  number_enrolled_ft           28685   
number_enrolled_total            number_enrolled_total           28526   
number_admitted                        number_admitted           28432   
number_applied                          number_applied           27863   
unitid                                    

/usr/local/lib/python3.10/dist-packages/missingno/missingno.py:366: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=figsize)


Summary of missing data for Feature Group 9:
                                                      Variable  \
hospital                                              hospital   
cc_basic_2000                                    cc_basic_2000   
medical_degree                                  medical_degree   
offering_highest_degree                offering_highest_degree   
degree_granting                                degree_granting   
urban_centric_locale                      urban_centric_locale   
hbcu                                                      hbcu   
tribal_college                                  tribal_college   
open_public                                        open_public   
open_public.1                                    open_public.1   
inst_status                                        inst_status   
year_deleted                                      year_deleted   
currently_active_ipeds                  currently_active_ipeds   
postsec_public_active          

<ipython-input-11-de87b5b52827>:67: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(figsize=(12, len(summary_df) * 0.3))


...

<Figure size 1000x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

# Step 3

In [ ]:
!pip install scikit-learn
!pip install tableone

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import pandas as pd
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt

columns_to_include = [
    'offering_highest_level', 'land_grant', 'inst_control', 'offering_undergrad',
    'offering_grad', 'urban_centric_locale', 'region', 'title_iv_indicator',
    'tribal_college', 'sector', 'hbcu', 'inst_size', 'number_enrolled_pt',
    'number_enrolled_ft', 'number_enrolled_total', 'number_admitted', 'number_applied',
    'living_arrangement', 'number_of_students', 'percent_of_students', 'academic_rank',
    'average_salary', 'avg_wgtd_mon_salary', 'instruc_staff_count', 'completers', 'race'
]
available_columns = [col for col in columns_to_include if col in merged_data.columns]
data_to_impute = merged_data[available_columns]

categorical_cols = [col for col in available_columns if col in [
    'offering_highest_level', 'land_grant', 'inst_control', 'offering_undergrad',
    'offering_grad', 'urban_centric_locale', 'region', 'title_iv_indicator',
    'tribal_college', 'sector', 'hbcu', 'inst_size', 'living_arrangement', 'race', 'academic_rank'
]]
numerical_cols = list(set(available_columns) - set(categorical_cols))

data_encoded = data_to_impute.copy()
for col in categorical_cols:
    if data_encoded[col].dtype == 'object' or str(data_encoded[col].dtype) == 'category':
        data_encoded[col] = data_encoded[col].astype('category').cat.codes

skewed_cols = ['number_enrolled_pt', 'number_enrolled_ft', 'number_enrolled_total', 'number_admitted', 'average_salary']
for col in skewed_cols:
    if col in data_encoded.columns:
        data_encoded[col] = np.log1p(data_encoded[col].clip(lower=0))

mice_imputer = IterativeImputer(max_iter=10, random_state=0)
imputed_array = mice_imputer.fit_transform(data_encoded)

imputed_data = pd.DataFrame(imputed_array, columns=available_columns)

for col in skewed_cols:
    if col in imputed_data.columns:
        imputed_data[col] = np.expm1(imputed_data[col])

imputed_data['number_enrolled_pt'] = imputed_data['number_enrolled_pt'].clip(lower=0, upper=1e6)
imputed_data['number_admitted'] = imputed_data['number_admitted'].clip(lower=0, upper=1e6)

summary_stats = []
for col in imputed_data.columns:
    col_data = imputed_data[col]
    if col in categorical_cols:
        mean_value = f"{(col_data.value_counts(normalize=True).max() * 100):.1f}%"
        min_value, max_value, std_value = "-", "-", "-"
    else:
        mean_value = f"{col_data.mean():,.2f}"
        min_value = f"{col_data.min():,.2f}"
        max_value = f"{col_data.max():,.2f}"
        std_value = f"{col_data.std():,.2f}"
    summary_stats.append([col, mean_value, min_value, max_value, std_value])

summary_df = pd.DataFrame(summary_stats, columns=["Characteristic", "Mean or %", "Minimum", "Maximum", "SD"])

with PdfPages("Summary_Table.pdf") as pdf:
    fig, ax = plt.subplots(figsize=(12, len(summary_df) * 0.3))
    ax.axis("tight")
    ax.axis("off")
    ax.table(cellText=summary_df.values,
             colLabels=summary_df.columns,
             cellLoc="center",
             loc="center")
    plt.title("Summary Table (MICE Imputed Data)", fontsize=14)
    pdf.savefig(fig)
    plt.close(fig)

from google.colab import files
files.download("Summary_Table.pdf")

print("Improved Summary Table created and downloaded as Summary_Table.pdf.")

/usr/local/lib/python3.10/dist-packages/sklearn/impute/_iterative.py:825: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Improved Summary Table created and downloaded as Summary_Table.pdf.
